# Trabalhando com SQL no Python

### Baixando o arquivo da aula:

In [1]:
import gdown

# URL do Google Drive convertida para download direto
url = 'https://drive.google.com/uc?id=1kcH4xji_A1_FCgHoDOf6eu9EYcX0blyP'

# Caminho local para salvar o arquivo .db
output_path = 'chinook.db'

# Baixar o arquivo
gdown.download(url, output_path, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1kcH4xji_A1_FCgHoDOf6eu9EYcX0blyP
To: e:\Senac\Aulas\CursoBigData\Semana-06\chinook.db
100%|██████████| 1.07M/1.07M [00:00<00:00, 2.57MB/s]


'chinook.db'

Chinook é um banco de dados de exemplo disponível para SQL Server, Oracle, MySQL, entre outros. Ele pode ser criado executando um único script SQL. O banco de dados Chinook é uma alternativa ao banco de dados Northwind, sendo ideal para demonstrações e testes de ferramentas ORM que visam servidores de banco de dados únicos e múltiplos.

### SQLite + Pandas 

Para trabalhar com o banco de dados em Python, primeiro use a função `connect` do `sqlite3` para se conectar ao banco de dados e obter um objeto `Connection`:

<img src="https://docs.yugabyte.com/images/sample-data/chinook/chinook-er-diagram.png" alt="Chinook sample database">

O pandas oferece um método eficiente para carregar dados de bancos de dados SQL. Geralmente, utilizamos esse método para executar uma consulta SQL, usando uma conexão já estabelecida com o banco de dados. Assim, para visualizar todas as tabelas do banco podemos utilizar a consulta:

O banco de dados possui onze tabelas. Analisando a tabela de Album, temos:

Perceba que temos duas colunas de índice. Para resolver isso:

Analisando outra tabela do banco:

Vamos criar uma função chamada sq para simplificar o nosso trabalho. Com ela não vamos precisar ficar escrevendo pd.read_sql a todo momento... 

Frequentemente selecionaremos linhas em um banco de dados que atendem a certos critérios de seleção, especialmente em grandes volumes de dados, onde um banco de dados pode conter muitas linhas. Apenas as linhas que satisfazem os critérios de seleção (formalmente chamados de predicados) são selecionadas. A cláusula WHERE do SQL especifica os critérios de seleção de uma consulta. Valores de string em consultas SQL são delimitados por aspas simples (').

Complicando um pouco:

A cláusula WHERE pode conter os operadores <, >, <=, >=, =, <> (diferente) e LIKE. O operador LIKE é usado para correspondência de padrões—procurando por strings que combinam com um padrão dado. Um padrão que contém o caractere curinga de porcentagem (%) procura por strings que tenham zero ou mais caracteres na posição do caractere de porcentagem no padrão. Por exemplo, vamos localizar todos os artistas cujo nome começa com a letra D:

A cláusula ORDER BY ordena os resultados de uma consulta em ordem ascendente (do menor para o maior) ou descendente (do maior para o menor), especificados com ASC e DESC, respectivamente. A ordem de classificação padrão é ascendente, portanto, ASC é opcional. Vamos ordenar os títulos dos álbuns em ordem ascendente:

Você pode mesclar dados de várias tabelas, o que é referido como juntar as tabelas, com o JOIN.

Para evitar a duplicação de colunas no seu resultado, você deve especificar explicitamente quais colunas deseja retornar na sua consulta e pode usar aliases para renomeá-las conforme necessário:

**Exercício 1:** Qual funcionário tem o maior número total de clientes?

**Exercício: 2** Quais são os principais clientes de acordo com o Invoice?

### Primeiro ETL

In [23]:
import requests
import sqlite3

def create_university_db(country):
    # URL da API
    url = f"http://universities.hipolabs.com/search?country={country}"
    
    # Fazer a requisição à API
    response = requests.get(url)
    universities = response.json()

    # Criar ou conectar ao banco de dados SQLite
    conn = sqlite3.connect('universities.db')
    c = conn.cursor()
    
    # Criar a tabela, se não existir
    c.execute('''
        CREATE TABLE IF NOT EXISTS universities (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            country TEXT,
            state_province TEXT,
            web_pages TEXT,
            domains TEXT
        );
    ''')

    # Inserir dados no banco de dados
    for university in universities:
        c.execute('''
            INSERT INTO universities (name, country, state_province, web_pages, domains)
            VALUES (?, ?, ?, ?, ?);
        ''', (
            university['name'],
            university['country'],
            university['state-province'],
            ', '.join(university['web_pages']),  # Convertendo listas em strings
            ', '.join(university['domains'])
        ))
    
    # Commitar as mudanças e fechar a conexão
    conn.commit()
    conn.close()

# Exemplo de uso
create_university_db("Brazil")


Conectando ao novo banco de dados:

Verificando as tabelas que foram geradas:

Acessando o banco das universidades:

Procurando todas as universidades de Pernambuco.